In [1]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp312-cp312-linux_x86_64.whl size=10855080 sha256=d96ada831ef36354f9512e5a202c686349ba6402e71a4c1f84b95e8b43bbc231
  Stored in directory: /root/.cache/pip/wheels/b2/00/4f/9ff8af07a0a53ac6007ea5d739da19cfe147a2df542b6899f8
Successfully built implicit


In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit import als
from implicit.evaluation import mean_average_precision_at_k
import os


/usr/local/lib/python3.12/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/tmp/pip-install-nkqw30uy/implicit_e6b541b2e2ae4343b26015413a54ee33/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [3]:
os.environ["MKL_NUM_THREADS"] = "1"

In [6]:
# Membaca dataset
train_df = pd.read_csv("/content/sample_data/train.csv")


In [7]:

# Menambahkan kolom interaction dengan nilai 1 (karena interaksi hanya 1)
train_df['interaction'] = 1


In [8]:
# Membuat mapping untuk user_id dan item_id
user_mapping = {user: idx for idx, user in enumerate(train_df['user_id'].unique())}
item_mapping = {item: idx for idx, item in enumerate(train_df['item_id'].unique())}


In [9]:
# Mengganti user_id dan item_id dengan indeks
train_df['user_idx'] = train_df['user_id'].map(user_mapping)
train_df['item_idx'] = train_df['item_id'].map(item_mapping)


In [10]:

# Membuat sparse matrix (user-item matrix)
sparse_matrix = csr_matrix((train_df['interaction'], (train_df['user_idx'], train_df['item_idx'])))


In [11]:
# Membangun model ALS
model = als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=30, random_state=42)

/usr/local/lib/python3.12/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [12]:

# Melatih model ALS
model.fit(sparse_matrix)


  0%|          | 0/30 [00:00<?, ?it/s]

In [13]:
def get_recommendations(user_id, model, sparse_matrix, num_recommendations=10):
    user_idx = user_id
    results = model.recommend(user_idx, sparse_matrix[user_idx], N=num_recommendations)

    if isinstance(results, tuple) and len(results) == 2:
        item_ids, scores = results
        recommended_items = list(item_ids)
    else:
        recommended_items = [item for item, score in results]

    return recommended_items


In [14]:
print(type(get_recommendations))
print(get_recommendations)


<class 'function'>
<function get_recommendations at 0x7ee0adce3060>


In [15]:
user_id = 1
recommended_items = get_recommendations(user_id, model, sparse_matrix, num_recommendations=10)
print(f"Rekomendasi untuk pengguna {user_id}: {recommended_items}")

Rekomendasi untuk pengguna 1: [np.int32(366), np.int32(351), np.int32(18), np.int32(154), np.int32(538), np.int32(1069), np.int32(131), np.int32(531), np.int32(2080), np.int32(138)]


SETELAH TUNING

In [21]:
def mean_average_precision(model, test_df, train_matrix, k=10, sample_users=100):
    users = test_df['user_idx'].unique()
    np.random.seed(42)

    if len(users) > sample_users:
        users = np.random.choice(users, size=sample_users, replace=False)

    aps = []

    for user in users:
        true_items = test_df[test_df['user_idx'] == user]['item_idx'].values

        # implicit version ini return 2 array (items, scores)
        items, scores = model.recommend(user, train_matrix[user], N=k)
        recommended_items = items

        hits = 0
        sum_precisions = 0.0

        for i, item in enumerate(recommended_items):
            if item in true_items:
                hits += 1
                sum_precisions += hits / (i+1)

        ap = sum_precisions / min(len(true_items), k)
        aps.append(ap)

    return np.mean(aps)


In [22]:
map_score = mean_average_precision(model, test_df_split, train_matrix, k=10, sample_users=100)
print(f"Mean Average Precision (MAP@10): {map_score:.4f}")


Mean Average Precision (MAP@10): 0.0175


MENINGKATKAN

In [26]:
from implicit.als import AlternatingLeastSquares

# BOOST WEIGHT
train_matrix_boost = train_matrix.copy()
train_matrix_boost.data = np.log1p(train_matrix_boost.data * 40)

model = AlternatingLeastSquares(
    factors=200,
    regularization=0.05,
    iterations=30,
    num_threads=8,
    use_gpu=False
)

model.fit(train_matrix_boost)

map_score = mean_average_precision(model, test_df_split, train_matrix_boost, k=10, sample_users=200)
print("MAP:", map_score)


  0%|          | 0/30 [00:00<?, ?it/s]

MAP: 0.01785846560846561
